In [1]:
import sys
sys.path.append('..')
from src.read_databases import read_database4, read_database3

In [4]:
read_database4(10,8, 8, 3, 'biceps', 'p', 10, None, pathcwd='../')

array([[-1.74238541e-02,  2.14170768e-02,  3.44896755e-02,
        -4.50036592e-03, -3.43323136e-02, -7.08993493e-03,
         5.13711521e-02,  2.49706775e-02],
       [-1.29892812e-02, -2.46039779e-02, -2.02811777e-02,
        -7.64778006e-03, -1.56966677e-02, -7.11257012e-03,
         1.82766340e-02, -1.46136311e-02],
       [-1.81030665e-03,  3.32862504e-03,  4.34225697e-03,
         4.20962506e-02,  1.50914041e-02, -1.02634883e-02,
        -4.06569783e-02, -6.16442909e-03],
       [ 1.76371176e-02, -3.24689040e-02,  3.53066145e-05,
         9.51738378e-03, -5.93914320e-03,  4.33618039e-03,
         4.72667448e-02,  3.00249670e-03],
       [-3.45200846e-02,  7.46027751e-03, -1.33752825e-03,
        -3.52369685e-02, -1.27939703e-02,  1.80674027e-02,
        -2.46302150e-02, -2.60679073e-03],
       [ 8.73712700e-03, -4.54696475e-02, -1.62257175e-02,
         2.31879515e-03,  2.31204809e-02, -1.60577621e-02,
        -3.35234515e-02, -3.84754949e-03],
       [ 2.58293965e-02, -2.442902

In [6]:
sys.path

['/media/rnwatanabe/Data/Renato/hd-EMG-dataset-merge',
 '/media/rnwatanabe/Data/Renato/hd-EMG-dataset-merge/notebooks',
 '/home/rnwatanabe/miniconda3/lib/python310.zip',
 '/home/rnwatanabe/miniconda3/lib/python3.10',
 '/home/rnwatanabe/miniconda3/lib/python3.10/lib-dynload',
 '',
 '/home/rnwatanabe/miniconda3/lib/python3.10/site-packages']

{'CHROME_DESKTOP': 'code-url-handler.desktop',
 'DBUS_SESSION_BUS_ADDRESS': 'unix:path=/run/user/1001/bus',
 'DEBUGINFOD_URLS': 'https://debuginfod.ubuntu.com ',
 'DEFAULTS_PATH': '/usr/share/gconf/ubuntu.default.path',
 'DESKTOP_SESSION': 'ubuntu',
 'DISPLAY': ':1',
 'GDK_BACKEND': 'x11',
 'GDMSESSION': 'ubuntu',
 'GIO_LAUNCHED_DESKTOP_FILE': '/usr/share/applications/code.desktop',
 'GIO_LAUNCHED_DESKTOP_FILE_PID': '14931',
 'GJS_DEBUG_OUTPUT': 'stderr',
 'GJS_DEBUG_TOPICS': 'JS ERROR;JS LOG',
 'GNOME_DESKTOP_SESSION_ID': 'this-is-deprecated',
 'GNOME_SHELL_SESSION_MODE': 'ubuntu',
 'GPG_AGENT_INFO': '/run/user/1001/gnupg/S.gpg-agent:0:1',
 'GTK2_MODULES': 'overlay-scrollbar',
 'GTK_MODULES': 'gail:atk-bridge',
 'HOME': '/home/rnwatanabe',
 'IM_CONFIG_PHASE': '1',
 'INVOCATION_ID': '01973f34134c482c9b942f9f170ba053',
 'JAVA_HOME': '/usr/lib/jvm/default-java',
 'JOURNAL_STREAM': '8:25318',
 'LANG': 'en_US.UTF-8',
 'LC_ADDRESS': 'en_US.UTF-8',
 'LC_IDENTIFICATION': 'en_US.UTF-8',
 'LC_M